In [6]:
import os


In [7]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install PyPDF2
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [76 lines of output]
      /workspaces/genai_mcq_generator/env/lib/python3.10/site-packages/setuptools/__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully

In [9]:
%pip install python-dotenv
%pip install openai


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [12]:
load_dotenv()

True

In [13]:
key=os.getenv("OPENAI_API_KEY")

In [14]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo-0125",temperature=0.7)

In [15]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f17ff979600>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f17ff97afe0>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')

In [27]:
with open("/workspaces/genai_mcq_generator/Response.json","r") as f:
    response_json=json.load(f)

In [28]:
print(response_json)

{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [34]:
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [42]:
quiz_gen_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template

)

In [43]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz",verbose=True)

In [44]:
new_template="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [53]:
quiz_review_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=new_template

)


In [54]:
print(quiz_review_prompt)

input_variables=['quiz', 'subject'] template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n'


In [55]:
quiz_review_chain=LLMChain(llm=llm,prompt=quiz_review_prompt,output_key="review",verbose=True)

In [57]:
genrate_eval_chain=SequentialChain(chains=[quiz_chain,quiz_review_chain],input_variables=["text", "number", "subject", "tone","response_json"],output_variables=["quiz", "review"],verbose=True,)

In [61]:
PATH="/workspaces/genai_mcq_generator/data.txt"

In [69]:
with open(PATH,'r') as file:
    text=file.read()

In [70]:
print(text)

Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is provided feedback that's analogous to rewards, which it tries to maximize.
Although each algorithm has advantage

In [73]:
with get_openai_callback() as cb:
    response=genrate_eval_chain(
        {"text":text,
        "number":3,
        "subject":"Machine Learning",
        "tone":"intermediate",
        "response_json":json.dumps(response_json)}
    )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is prov


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Machine Learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

{
"1": {
"mcq": "Which machine learning approach involves the computer being presented with example inputs and their desired outputs?",
"options": {
"a": "Supervised learning",
"b": "Unsupervised learning",
"c": "Reinforcement learning",
"d": "Semi-supervised learning"
},
"correct": "a"
},
"2": {
"mcq": "What type of learning algorithm has no labels given to the learning algorithm, allowing it to find structure in its input?",
"options": {
"a"

In [77]:
print(response["review"])
print(response["quiz"])

The complexity of the quiz is moderate as it requires understanding of basic concepts in machine learning. The questions are clear and concise, suitable for machine learning students. No changes needed.

{
"1": {
"mcq": "Which machine learning approach involves the computer being presented with example inputs and their desired outputs?",
"options": {
"a": "Supervised learning",
"b": "Unsupervised learning",
"c": "Reinforcement learning",
"d": "Semi-supervised learning"
},
"correct": "a"
},
"2": {
"mcq": "What type of learning algorithm has no labels given to the learning algorithm, allowing it to find structure in its input?",
"options": {
"a": "Supervised learning",
"b": "Unsupervised learning",
"c": "Reinforcement learning",
"d": "Semi-supervised learning"
},
"correct": "b"
},
"3": {
"mcq": "In which machine learning approach does a computer program interact with a dynamic environment and try to maximize rewards?",
"options": {
"a": "Supervised learning",
"b": "Unsupervised learning"

In [78]:
quiz=json.loads(response.get("quiz"))

In [79]:
quiz

{'1': {'mcq': 'Which machine learning approach involves the computer being presented with example inputs and their desired outputs?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Semi-supervised learning'},
  'correct': 'a'},
 '2': {'mcq': 'What type of learning algorithm has no labels given to the learning algorithm, allowing it to find structure in its input?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Semi-supervised learning'},
  'correct': 'b'},
 '3': {'mcq': 'In which machine learning approach does a computer program interact with a dynamic environment and try to maximize rewards?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Semi-supervised learning'},
  'correct': 'c'}}

In [82]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [83]:
quiz_table_data

[{'MCQ': 'Which machine learning approach involves the computer being presented with example inputs and their desired outputs?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Semi-supervised learning',
  'Correct': 'a'},
 {'MCQ': 'What type of learning algorithm has no labels given to the learning algorithm, allowing it to find structure in its input?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Semi-supervised learning',
  'Correct': 'b'},
 {'MCQ': 'In which machine learning approach does a computer program interact with a dynamic environment and try to maximize rewards?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Semi-supervised learning',
  'Correct': 'c'}]

In [85]:
df=pd.DataFrame(quiz_table_data)

In [86]:
df.to_csv("ML_quiz.xlsx")